## 初始化

In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

In [ ]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='test1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
pprint(project.tender_file_extraction)
cache.save_json('招标文件/tender_file_extraction', project.tender_file_extraction)

## PART-1. 招标文件 大纲分析


### 1.1 分析第一层级标题
输入： tender_file_extraction
输出： title_lists_H1， tender_file_extraction_H1


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction = cache.load_json('招标文件/tender_file_extraction')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction)
print(headings)

In [15]:
# LLM 数据准备
# 输入： tender_file_extraction, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TendeorOutlinesL1 里的 prepare_context 调用 TiptapUtils.extract_indexed_paragraphs
from apps.projects.services.tasks_preparation.tender_outlines_L1 import TenderOutlinesL1
# #获取LLM分析相关数据
data_cooker_l1 = TenderOutlinesL1(tender_file_extraction)
model_params_l1, task_l1, meta_l1 = data_cooker_l1.output_params()
# # 模拟的prompt, 
simulated_prompt, formatted_message = data_cooker_l1.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/model_params_l1', model_params_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/task_l1', task_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/meta_l1', meta_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/formatted_message', formatted_message)


In [ ]:
# LLM分析
# 输入：model_params, task_l1,  输出：针对tender_file_extraction的H1层级标题列表 （json格式）
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_l1)
# 大模型输出的是json str格式，可能带有标记噪音
result = await analyzer.process(task_l1)
# 清除标记，转为json dict格式
result_clean_parsed = clean_llm_JSON_output(result)

print(f'result类型： {type(result)}, cleaned_result类型：{type(result_clean_parsed)}')
pprint(result_clean_parsed)

cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/result', result)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/result_clean_parsed', result_clean_parsed)

In [ ]:
# 更新文档标题
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H1 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction, 
    title_list=result_clean_parsed, 
    index_path_map=meta_l1["index_path_map"]
    )

cache.save_json('招标文件/tender_file_extraction_H1', tender_file_extraction_H1)
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H1)
print(TiptapUtils.print_headings(tender_file_extraction_H1))

### 1.2 分析第二、三层级标题
输入： tender_file_extraction_H1， 输出：title_lists_H2/3, tender_file_extraction_H23


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H1 =cache.load_json('招标文件/tender_file_extraction_H1')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H1)
print(headings)


In [18]:
# LLM 数据准备
# 输入： tender_file_extraction_H1, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.tasks_preparation.tender_outlines_L2 import TenderOutlinesL2
# #获取LLM分析相关数据
data_cooker_l2 = TenderOutlinesL2(tender_file_extraction_H1)
model_params_l2, tasks_l2, meta_l2 = data_cooker_l2.output_params()
# # 模拟的prompt, 
simulated_prompts_l2, formatted_messages_l2 = data_cooker_l2.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/model_params_l2', model_params_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/tasks_l2', tasks_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/meta_l2', meta_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/formatted_message', formatted_messages_l2)

In [ ]:
# LLM batch分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_l2)
results = await analyzer.process_with_limit(tasks_l2, limit=5)

results_clean_parsed = process_concurrent_JSON_outputs(results)
print(f'result类型： {type(results)}, cleaned_result类型：{type(results_clean_parsed)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/results', results)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/results_clean_parsed', results_clean_parsed)

In [ ]:
# LLM 结果修正
# 将所有的'level'值加1
for item in results_clean_parsed:
    if 'level' in item:
        item['level'] += 1

from pprint import pprint
pprint(results_clean_parsed)

In [ ]:
# 更新文档标题 H2-3
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H2 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction_H1, 
    title_list=results_clean_parsed, 
    index_path_map=meta_l2["index_path_map"]
    )

# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H2)
from pprint import pprint
#pprint(headings)
print(TiptapUtils.print_headings(tender_file_extraction_H2))

# 存储tender_file_extraction_L2 结果
cache.save_json('招标文件/tender_file_extraction_H2', tender_file_extraction_H2)

### 1.3 添加前言目录
输入： tender_file_extraction   输出: tender_file_extraction_final

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H2 =cache.load_json('招标文件/tender_file_extraction_H2')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H2)
print(headings)

In [ ]:
# 直接添加前言
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final = TiptapUtils.add_introduction_headings(tender_file_extraction_H2)
pprint(tender_file_extraction_final)
cache.save_json('招标文件/tender_file_extraction_final', tender_file_extraction_final)

In [ ]:
#打印 带前言的文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final)
print(enhanced_toc)

### 1.4 增强目录分析（图表）

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final =cache.load_json('招标文件/tender_file_extraction_final')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_final)
print(headings)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.tasks_preparation.tender_outlines_tables import TenderOutlinesTables
# #获取LLM分析相关数据
data_cooker_tb = TenderOutlinesTables(tender_file_extraction_final)
model_params_tb, tasks_tb, meta_tb = data_cooker_tb.output_params()
# # 模拟的prompt, 
simulated_prompts_tb, formatted_messages_tb = data_cooker_tb.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/model_params_tb', model_params_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/tasks_tb', tasks_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/meta_tb', meta_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/formatted_messages_tb', formatted_messages_tb)

In [ ]:
# LLM batch分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_tb)
results_tb = await analyzer.process_with_limit(tasks_tb, limit=5)
pprint(results_tb)

results_tb_clean_parsed = process_concurrent_JSON_outputs(results_tb)
print(f'result类型： {type(results_tb)}, cleaned_result类型：{type(results_tb_clean_parsed)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results', results_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results_clean_parsed', results_tb_clean_parsed)

In [ ]:
results_tb_clean_parsed = cache.load_json('t1-招标文件大纲分析/t1.4-增强标题分析/results_clean_parsed')
meta_tb = cache.load_json('t1-招标文件大纲分析/t1.4-增强标题分析/meta_tb')
pprint(results_tb_clean_parsed)
pprint(meta_tb)

In [ ]:
# 更新文档，给节点添加caption note
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final_tb = TiptapUtils.add_captions_to_nodes(
    doc = tender_file_extraction_final, 
    captions = results_tb_clean_parsed, 
    index_path_map = meta_tb["index_path_map"]
    )
pprint(tender_file_extraction_final_tb)
cache.save_json('招标文件/tender_file_extraction_final_tb', tender_file_extraction_final_tb)

In [ ]:
#打印 增加型文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(enhanced_toc)

## PART-2. 投标文件编辑格式和要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标文件组成"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topic_context_batch import FindTopicContextBatch
# #获取LLM分析相关数据
data_cooker_bid = FindTopicContextBatch(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit(tasks_bid, limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标文件组成框架分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：list[]投标文件目录框架

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed')
paths = [item['path'] for item in relavant_parts]
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.bid_outlines import BidOutlines
# #获取LLM分析相关数据
data_cooker_bid = BidOutlines(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/tasks_bid', task_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
pprint(results_clean_parsed)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results_clean_parsed', results_clean_parsed)

## PART-3. 投标人资格要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标人资格要求"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topic_context_batch import FindTopicContextBatch
# #获取LLM分析相关数据
data_cooker_bid = FindTopicContextBatch(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit(tasks_bid, limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results', results)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标人资格要求分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：？？？

In [1]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed')
paths = [item['path'] for item in relavant_parts]
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

[[53], [359]]


In [ ]:
print(headings)

In [2]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.s1_bidder_qulification.qualification_analysis import QualificationAnalysis
# #获取LLM分析相关数据
data_cooker_bid = QualificationAnalysis(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/tasks_bid', task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

WARNING 2025-04-24 07:33:56,546 helpers 在路径 [359] 处未找到标题节点


{'context': '## 六、合格投标人的资格要求(本项目采用资格后审)\n'
            '\n'
            '（1）在中华人民共和国境内注册的、具有独立承担民事责任的能力；\n'
            '\n'
            '（2）具有依法缴纳税收和社会保险的良好记录；\n'
            '\n'
            '（3）近三年内，在经营活动中没有重大违法记录；\n'
            '\n'
            '（4）单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得 同时参加本项目的投标；\n'
            '\n'
            '（5）信用中国网站（www.creditchina.gov.cn）和中国政府采购网\n'
            '\n'
            '（www.ccgp.gov.cn）查询信用记录中（查询时间为招标公告发布后至投标截止时 '
            '间期间），被列入失信被执行人、重大税收违法案件当事人名单或政府采购严重 违法失信行为记录名单的投标人，不得参加本项目投标；\n'
            '\n'
            '（6）具有有效的《食品生产许可证》或《食品经营许可证》。若投标人为仅 '
            '销售预包装食品的企业，须具有包含“食品经营（仅销售预包装食品）”的有效 营业执照或仅销售预包装食品备案证明材料；\n'
            '\n'
            '（7）投标人财务状况正常，须提供会计师事务所出具的上一年度（2023年度） '
            '财务审计报告，投标人尚未完成2023年度财务审计的须提供2022年度经审计的财\n'
            '\n'
            '务报告；若投标人为2023年1月1日之后成立的公司，须提供财务状况说明；\n'
            '\n'
            '（8）法律、行政法规规定的其他条件；\n'
            '\n'
            '（9）本项目不接受联合体投标。\n'
            '\n'
            '\n'

In [3]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/results', results)

HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
```markdown
| 编号 | 要求要点 | 所需证明材料 | 准备工作建议 | 我方是否满足 |
|------|----------|--------------|--------------|--------------|
| 1    | 在中华人民共和国境内注册，具有独立承担民事责任的能力 | 营业执照副本复印件或其他注册证明文件 | 确保公司已完成工商注册，并核实营业执照是否在有效期内 |              |
| 2    | 具有依法缴纳税收和社会保险的良好记录 | 最近六个月的税收缴纳证明、社会保险缴纳证明 | 收集并整理最近六个月的税务及社保缴纳凭证，确保无欠缴或异常记录 |              |
| 3    | 近三年内，在经营活动中没有重大违法记录 | 相关声明文件或第三方机构出具的无违法记录证明 | 核查公司近三年的经营活动，确保无重大违法记录；必要时可向相关部门申请证明 |              |
| 4    | 单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得同时参加本项目的投标 | 投标人关联关系声明 | 梳理公司股东结构及管理层信息，避免与关联企业同时参与投标 |              |
| 5    | 未被列入失信被执行人、重大税收违法案件当事人名单或政府采购严重违法失信行为记录名单 | 信用中国网站和中国政府采购网查询结果截图（查询时间需在招标公告发布后至投标截止时间期间） | 在规定时间内完成信用记录查询，并保存查询结果的截图作为证明材料 |              |
| 6    | 具有有效的《食品生产许可证》或《食品经营许可证》；若为仅销售预包装食品的企业，须提供相关证明 | 《食品生产许可证》或《食品经营许可证》复印件；若为仅销售预包装食品的企业，还需提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明 | 核实公司是否持有相关许可证，并确保证件在有效期内；若为新办企业，及时办理相关资质 |              |
| 7    | 财务状

In [4]:
from IPython.display import display, HTML, Markdown
display(Markdown(results))

```markdown
| 编号 | 要求要点 | 所需证明材料 | 准备工作建议 | 我方是否满足 |
|------|----------|--------------|--------------|--------------|
| 1    | 在中华人民共和国境内注册，具有独立承担民事责任的能力 | 营业执照副本复印件或其他注册证明文件 | 确保公司已完成工商注册，并核实营业执照是否在有效期内 |              |
| 2    | 具有依法缴纳税收和社会保险的良好记录 | 最近六个月的税收缴纳证明、社会保险缴纳证明 | 收集并整理最近六个月的税务及社保缴纳凭证，确保无欠缴或异常记录 |              |
| 3    | 近三年内，在经营活动中没有重大违法记录 | 相关声明文件或第三方机构出具的无违法记录证明 | 核查公司近三年的经营活动，确保无重大违法记录；必要时可向相关部门申请证明 |              |
| 4    | 单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得同时参加本项目的投标 | 投标人关联关系声明 | 梳理公司股东结构及管理层信息，避免与关联企业同时参与投标 |              |
| 5    | 未被列入失信被执行人、重大税收违法案件当事人名单或政府采购严重违法失信行为记录名单 | 信用中国网站和中国政府采购网查询结果截图（查询时间需在招标公告发布后至投标截止时间期间） | 在规定时间内完成信用记录查询，并保存查询结果的截图作为证明材料 |              |
| 6    | 具有有效的《食品生产许可证》或《食品经营许可证》；若为仅销售预包装食品的企业，须提供相关证明 | 《食品生产许可证》或《食品经营许可证》复印件；若为仅销售预包装食品的企业，还需提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明 | 核实公司是否持有相关许可证，并确保证件在有效期内；若为新办企业，及时办理相关资质 |              |
| 7    | 财务状况正常，须提供会计师事务所出具的上一年度财务审计报告；若尚未完成2023年度审计，须提供2022年度审计报告；若为2023年1月1日之后成立的公司，须提供财务状况说明 | 上一年度（2023年度）或2022年度财务审计报告；若为新成立公司，提供财务状况说明 | 提前联系会计师事务所完成财务审计，或准备详细的财务状况说明文件 |              |
| 8    | 符合法律、行政法规规定的其他条件 | 相关法律法规要求的证明文件 | 查阅与本项目相关的法律法规，确保符合所有规定，并准备相应证明材料 |              |
| 9    | 本项目不接受联合体投标 | 无 | 确保以单一主体身份参与投标，避免与其他单位组成联合体 |              |
``` 

以上表格逐条整理了投标人资格要求的要点、所需证明材料及准备工作建议，便于用户后续填写“我方是否满足”一栏。